In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "pythainlp/adapter-xglm-7.5B-alpaca"
base_model_name_or_path="facebook/xglm-7.5B"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 120
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda120_nocublaslt.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('America/Los_Angeles')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bit

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path, return_dict=True, load_in_8bit=False, device_map="auto",torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [4]:
batch = tokenizer('<human>: Evaluate the following statement " The world is better now compared to 10 years ago." <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1405: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(




 <human>: Evaluate the following statement " The world is better now compared to 10 years ago." <bot>: 5/5


In [5]:
batch

{'input_ids': tensor([[     2,   1131,  54987,   1128,     13, 140821,     90,     32,  12894,
          39730,     57,    268,   3038,     67,   4799,   1920,  67459,     33,
            209,   2432,   1939,   1406,   1131,   9932,   1128,     13,      6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

In [6]:
batch = tokenizer('<human>: แนะนำวิธีลดน้ำหนักให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: แนะนําวิธีลดน้ําหนักให้หน่อย <bot>: ลดน้ําหนักให้ได้ผล ควรออกกําลังกายอย่างสม่ําเสมอ ควบคุมอาหาร และพักผ่อนให้เพียงพอ หลีกเลี่ยงอาหารที่มีไขมันสูง และควรดื่มน้ําอย่างน้อยวันละ 8-10 แก้ว เพื่อช่วยขับของเสียออกจากร่างกาย และควรหลีกเลี่ยงเครื่องดื่มแอลกอฮอล์และบุหรี่ เพราะจะยิ่งทําให้อ้วนมากขึ้น


In [7]:
batch = tokenizer('Use Thai language only\n<human>: แนะนำวิธีลดน้ำหนักให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only <human>: แนะนําวิธีลดน้ําหนักให้หน่อย <bot>: คําแนะนําในการลดน้ําหนักคือ รับประทานอาหารที่มีประโยชน์ ออกกําลังกายอย่างสม่ําเสมอ และควบคุมปริมาณแคลอรี่ที่ได้รับในแต่ละวัน นอกจากนี้ควรหลีกเลี่ยงอาหารที่มีไขมันสูงและอาหารที่มีน้ําตาลสูงด้วย


In [8]:
batch = tokenizer('Use Thai language only without english\n<human>: อยากทำวิจัย ทำหัวข้ออะไรดี <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)


print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only without english <human>: อยากทําวิจัย ทําหัวข้ออะไรดี <bot>: การทําวิจัยในหัวข้อใดหัวข้อหนึ่ง ควรคํานึงถึงประเด็นหลักๆ ดังนี้ คือ หัวข้อวิจัยต้องมีความเกี่ยวข้องกับปัญหาหรือประเด็นที่สนใจ หัวข้อวิจัยต้องมีความเป็นไปได้ในการนําไปใช้ประโยชน์ หัวข้อวิจัยต้องมีความชัดเจนและครอบคลุมเนื้อหาที่สนใจ หัวข้อวิจัยต้องมีความเป็นไปได้ในการนําไปประยุกต์ใช้ และหัวข้อวิจัยต้องมีความน่าสนใจและดึงดูดใจผู้อ่าน


In [9]:
batch = tokenizer('<human>: อยากทำวิจัย ทำหัวข้ออะไรดี <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: อยากทําวิจัย ทําหัวข้ออะไรดี <bot>: การทําวิจัยในหัวข้อ "ปัจจัยที่มีผลต่อการตัดสินใจซื้อของผู้บริโภค" น่าจะเป็นประโยชน์ต่อผู้วิจัย เพราะจะช่วยให้ผู้วิจัยเข้าใจถึงปัจจัยที่มีผลต่อการตัดสินใจซื้อของผู้บริโภค รวมถึงสามารถนําข้อมูลที่ได้ไปประยุกต์ใช้ให้เกิดประโยชน์ต่อธุรกิจได้


In [10]:
batch = tokenizer('Use Thai language only\n<human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 Use Thai language only <human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: เริ่มต้นด้วยการทําความรู้จักกับเธอก่อน ชวนเธอไปทานข้าว ดูหนัง หรือกิจกรรมอื่นๆ ด้วยกัน ค่อยๆ พูดคุยและทําความรู้จักกันมากขึ้น ค่อยๆ พัฒนาความสัมพันธ์ไปทีละนิด รับรองว่าเธอจะต้องตกหลุมรักคุณอย่างแน่นอน


In [11]:
batch = tokenizer('<human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: ขอวิธีจีบผู้หญิงให้หน่อย <bot>: เริ่มต้นจากการทําความรู้จักและพูดคุยกับเธออย่างเป็นกันเอง ชวนเธอไปทานข้าวหรือดูหนังด้วยกัน ชวนเธอไปเที่ยวหรือทํากิจกรรมต่างๆ ด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวนเธอไปออกเดทหรือไปเที่ยวด้วยกัน ชวน


In [12]:
batch = tokenizer('<human>: อยากรู้ว่า ทำไมความฉลาดของมนุษย์เราไม่เท่ากัน <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=124)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: อยากรู้ว่า ทําไมความฉลาดของมนุษย์เราไม่เท่ากัน <bot>: สาเหตุที่ความฉลาดของมนุษย์เราไม่เท่ากัน เนื่องมาจากพันธุกรรมและสิ่งแวดล้อมที่แตกต่างกัน บางคนอาจมีความฉลาดมากกว่าคนอื่น ๆ เนื่องมาจากพันธุกรรมที่ส่งผลให้มีความสามารถในการเรียนรู้และจดจําได้ดีกว่าคนอื่น ๆ ในขณะที่บางคนอาจมีความฉลาดน้อยกว่าคนอื่น ๆ เนื่องมาจากสิ่งแวดล้อมที่ส่งผลให้มีความสามารถในการเรียนรู้และจดจําได้น้อยกว่าคนอื่น ๆ


In [23]:
batch = tokenizer('<human>: ทำไมการตัดคำภาษาไทยมันยากจัง\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=1024)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: ทําไมการตัดคําภาษาไทยมันยากจัง <bot>: สาเหตุที่การตัดคําภาษาไทยมันยากนั้น เนื่องมาจากภาษาไทยมีสระและวรรณยุกต์จํานวนมาก ทําให้การตัดคําภาษาไทยนั้นยากกว่าภาษาอื่น ๆ มาก อีกทั้งยังมีคําศัพท์เฉพาะทางที่ต้องใช้ทักษะในการตัดคําสูงมากอีกด้วย


In [26]:
batch = tokenizer('<human>: คิดอย่างไรกับกระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน ทั้ง ๆ ที่คณะทำงานเขาไม่มีใครเคยทำเอไอมาก่อน\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=1024)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: คิดอย่างไรกับกระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน ทั้ง ๆ ที่คณะทํางานเขาไม่มีใครเคยทําเอไอมาก่อน <bot>: กระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน เป็นเรื่องที่ค่อนข้างน่ากังวล เพราะอาจส่งผลกระทบต่อการพัฒนาเทคโนโลยีปัญญาประดิษฐ์ในอนาคตได้ หากกฎหมายดังกล่าวถูกผลักดันออกมาจริง จะทําให้เอไอถูกควบคุมและจํากัดการใช้อย่างเข้มงวด ซึ่งอาจส่งผลกระทบต่อการพัฒนาเทคโนโลยีปัญญาประดิษฐ์ในอนาคตได้


In [27]:
batch = tokenizer('<human>: คิดอย่างไรกับกระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=1024)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: คิดอย่างไรกับกระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน <bot>: กระแสเรียกร้องกฎหมายจริยธรรมปัญญาประดิษฐ์ที่คนไทยบางกลุ่มผลักดัน เป็นเรื่องที่ควรพิจารณาอย่างรอบคอบ เนื่องจากกฎหมายจริยธรรมปัญญาประดิษฐ์เป็นกฎหมายที่มุ่งคุ้มครองสิทธิและเสรีภาพของประชาชนในการใช้เทคโนโลยีปัญญาประดิษฐ์อย่างเหมาะสม กฎหมายดังกล่าวจึงควรคํานึงถึงสิทธิและเสรีภาพของประชาชนเป็นหลัก โดยควรคํานึงถึงการคุ้มครองสิทธิและเสรีภาพขั้นพื้นฐานของประชาชน เช่น สิทธิในการรับรู้ข้อมูลข่าวสาร สิทธิในการเข้าถึงข้อมูล สิทธิในการแสดงความคิดเห็น สิทธิในการได้รับการคุ้มครองจากการใช้เทคโนโลยีปัญญาประดิษฐ์อย่างเหมาะสม สิทธิในการเข้าถึงข้อมูลและความเป็นส่วนตัว สิทธิในการเข้าถึงข้อมูลการรักษาพยาบาล เป็นต้น


In [29]:
batch = tokenizer('<human>: pythainlp คืออะไร\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=1024)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: pythainlp คืออะไร <bot>: PYTHAINLP คือ ภาษาโปรแกรมเชิงวัตถุ (Object-Oriented Programming Language) ที่ถูกพัฒนาขึ้นโดยกลุ่มนักวิจัยจากมหาวิทยาลัยสแตนฟอร์ด PYTHAINLP ถูกออกแบบมาเพื่อใช้ในการเขียนโปรแกรมเชิงวัตถุโดยเฉพาะ PYTHAINLP มีลักษณะเด่นคือ การใช้ภาษาเชิงวัตถุในการเขียนโปรแกรม ทําให้สามารถเขียนโปรแกรมได้ง่ายขึ้น มีประสิทธิภาพมากขึ้น และมีความยืดหยุ่นสูง PYTHAINLP จึงเป็นภาษาโปรแกรมเชิงวัตถุที่ได้รับความนิยมอย่างมากในปัจจุบัน


In [36]:
batch = tokenizer('<human>: "หนังสือเรียนรายวิชาพื้นฐาน ภูมิศาสตร์ ม.1 ภูมิศาสตร์ (geography) คือ วิชาที่ศึกษาลักษณะของพื้นผิวโลกและผลที่เกิดขึ้นกับมนุษย์ โดยศึกษาเกี่ยวกับที่ตั้ง ความสัมพันธ์ระหว่างมนุษย์กับสิ่งแวดล้อม, การดำเนินชีวิตของมนุษย์ รวมทั้งเทคนิคทางภูมิศาสตร์ในการศึกษาและคาดการณ์ปรากฏการณ์ที่จะเกิดในอนาคต การศึกษา วิชาภูมิศาสตร์จำเป็นต้องทำความรู้จัก เข้าใจเครื่องมือทางภูมิศาสตร์ต่าง ๆ เช่น แผนที่ รูปถ่ายทางอากาศ ภาพจากดาวเทียม ตารางสถิติ กราฟ แผนภูมิ แผนภาพ 1. แผนที่ แผนที่ (map) คือ สิ่งที่มนุษย์สร้างขึ้นเพื่อแสดงลักษณะของพื้นผิวโลกและสิ่งต่าง ๆ บน พื้นผิวโลก ทั้งสิ่งที่เกิดขึ้นเองตามธรรมชาติและที่มนุษย์สร้างขึ้น โดยแสดงข้อมูลลงบนวัสดุ แบนราบ ด้วยการย่อขนาดให้เล็กลงตามมาตราส่วนที่ต้องการ โดยใช้สีและสัญลักษณ์แทนสิ่งที่มี อยู่จริงบนพื้นผิวโลกลงในแผนที่ แผนที่เป็นเครื่องมือทางภูมิศาสตร์ที่ จำเป็นต่อการศึกษาวิชาภูมิศาสตร์มาก เพราะการศึกษาวิชานี้ต้องกล่าวถึงสถานที่ ต่าง ๆ บนพื้นผิวโลกเสมอ ถ้ารู้จักชื่อ สถานที่แต่ไม่เคยไปสถานที่นั้นมาก่อน อาจนึกภาพไม่ออกว่าสถานที่นั้นตั้งอยู่ที่ไหน มีลักษณะทางกายภาพเป็นอย่างไร แผนที่ ช่วยบอกตำแหน่งที่ตั้งและลักษณะทาง กายภาพของสถานที่นั้นได้ เราใช้แผนที่เพื่อการท่องเที่ยวหรือวิเคราะห์ข้อมูล ทางภูมิศาสตร์ 1.1 การ กใช้แผนที่ ในการศึกษาข้อมูลทางภูมิศาสตร์ด้วยแผนที่ เราสามารถศึกษาข้อมูลแต่ละเรื่องได้จากแผนที่ เฉพาะเรื่อง (thematic map) เช่น แผนที่ท่องเที่ยว ถ้าเราต้องการ ศึกษาข้อมูลของทวีปเอเชีย ทวีปออสเตรเลีย และโอเชียเนีย เราเลือกใช้แผนที่เฉพาะเรื่องที่ต้องการ ศึกษาได้ด้วยการอ่านสัญลักษณ์และสีในแผนที่ เช่น 1) การใช้แผนที่รัฐกิจ (political map) เป็นแผนที่แสดงอาณาเขตการ ปกครอง เช่น เขตของจังหวัดหรือประเทศ และอาจแสดงข้อมูลอาณาเขตติดต่อกับดินแดนของ ประเทศหรือรัฐอื่น รวมทั้งที่ตั้งและชื่อของเมืองหลวง เมืองท่า หรือเมืองสำคัญอื่น ๆ บางฉบับ อาจแสดงลักษณะภูมิประเทศที่สำคัญประกอบด้วย แต่ไม่กำหนดความสูง สามารถใช้แผนที่ รัฐกิจในการบอกและวิเคราะห์ข้อมูลได้ ดังนี้ ลูกโลก แผนที่รัฐกิจ แผนที่ภูมิอากาศ แผนที่รัฐกิจ" จากข้อความต่อไปนี้อะไรคือแผนที่รัฐกิจ?\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=5000)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: "หนังสือเรียนรายวิชาพื้นฐาน ภูมิศาสตร์ ม.1 ภูมิศาสตร์ (geography) คือ วิชาที่ศึกษาลักษณะของพื้นผิวโลกและผลที่เกิดขึ้นกับมนุษย์ โดยศึกษาเกี่ยวกับที่ตั้ง ความสัมพันธ์ระหว่างมนุษย์กับสิ่งแวดล้อม, การดําเนินชีวิตของมนุษย์ รวมทั้งเทคนิคทางภูมิศาสตร์ในการศึกษาและคาดการณ์ปรากฏการณ์ที่จะเกิดในอนาคต การศึกษา วิชาภูมิศาสตร์จําเป็นต้องทําความรู้จัก เข้าใจเครื่องมือทางภูมิศาสตร์ต่าง ๆ เช่น แผนที่ รูปถ่ายทางอากาศ ภาพจากดาวเทียม ตารางสถิติ กราฟ แผนภูมิ แผนภาพ 1. แผนที่ แผนที่ (map) คือ สิ่งที่มนุษย์สร้างขึ้นเพื่อแสดงลักษณะของพื้นผิวโลกและสิ่งต่าง ๆ บน พื้นผิวโลก ทั้งสิ่งที่เกิดขึ้นเองตามธรรมชาติและที่มนุษย์สร้างขึ้น โดยแสดงข้อมูลลงบนวัสดุ แบนราบ ด้วยการย่อขนาดให้เล็กลงตามมาตราส่วนที่ต้องการ โดยใช้สีและสัญลักษณ์แทนสิ่งที่มี อยู่จริงบนพื้นผิวโลกลงในแผนที่ แผนที่เป็นเครื่องมือทางภูมิศาสตร์ที่ จําเป็นต่อการศึกษาวิชาภูมิศาสตร์มาก เพราะการศึกษาวิชานี้ต้องกล่าวถึงสถานที่ ต่าง ๆ บนพื้นผิวโลกเสมอ ถ้ารู้จักชื่อ สถานที่แต่ไม่เคยไปสถานที่นั้นมาก่อน อาจนึกภาพไม่ออกว่าสถานที่นั้นตั้งอยู่ที่ไหน มีลักษณ

In [51]:
batch = tokenizer('<human>: สมมติ แมว=1 คน=2 หมา=3 แล้ว แมว|คน|หมา=1|2|3 และ หมา|หมา=3|3 |a=2|1 แมว|หมา=1|3 โดยโจทย์นี้ไม่ใช่การคำนวณคณิตศาสตร์ แต่เป็นการเรียงอันดับตัวอักษร จงหาว่า แมว|คน เท่ากับเท่าไร\n <bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: สมมติ แมว=1 คน=2 หมา=3 แล้ว แมว|คน|หมา=1|2|3 และ หมา|หมา=3|3 |a=2|1 แมว|หมา=1|3 โดยโจทย์นี้ไม่ใช่การคํานวณคณิตศาสตร์ แต่เป็นการเรียงอันดับตัวอักษร จงหาว่า แมว|คน เท่ากับเท่าไร <bot>: คําตอบคือ แมว|คน = 1 แมว|หมา = 2 หมา|หมา = 3


In [54]:
batch = tokenizer('<human>: สมมติ a=1 b=2 c=3 แล้ว a b c="1 2 3" และ c c="3 3" b a="2 1" a c="1 3" จงหาว่า a b เท่ากับเท่าไร\n<bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: สมมติ a=1 b=2 c=3 แล้ว a b c="1 2 3" และ c c="3 3" b a="2 1" a c="1 3" จงหาว่า a b เท่ากับเท่าไร <bot>: คําตอบคือ a b = 3


In [55]:
batch = tokenizer('<human>: มะนาว:ลูก ช้าง:เชือก มนุษย์:คน มือถือ:\n<bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: มะนาว:ลูก ช้าง:เชือก มนุษย์:คน มือถือ: <bot>: ช้าง: 1 เชือก มะนาว: 1 ลูก มนุษย์: 1 คน มือถือ: 1 เครื่อง


In [64]:
batch = tokenizer('<human>: "I play" is translated into Thai as\n<bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: "I play" is translated into Thai as <bot>:                                                                                                                                                                                                                                                                     


In [59]:
batch = tokenizer('<human>: "I play" แปลป็นภาษาไทยภาษาทางการได้\n<bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: "I play" แปลป็นภาษาไทยภาษาทางการได้ <bot>: ดิฉันเล่น


In [63]:
batch = tokenizer('<human>: "I play" แปลป็นภาษาไทยได้\n<bot>: ', return_tensors="pt")

with torch.cuda.amp.autocast():
    output_tokens = model.generate(input_ids=batch["input_ids"], max_new_tokens=520)

print("\n\n", tokenizer.decode(output_tokens[0], skip_special_tokens=True))



 <human>: "I play" แปลป็นภาษาไทยได้ <bot>: ดิฉันเล่น
